# Timetable

The primary objective of this script is to gather precise arrival timings for both buses and trams at every stop located in Warsaw. Executing the script on a particular day will provide the timetable for that specific day. For instance, in this case, the data was collected on the 16th of January 2023, and thus the obtained timetable is valid only for this day.

Importing the needed libraries

In [1]:
import urllib.request, json
import pandas as pd
import requests

Data regarding all the stops in Warsaw is gathered through the API access point (https://api.um.warszawa.pl/). The gathered data is then stored in the df_loc DataFrame. The essential information extracted from the API includes:
1. Stop number ('zespol')
2. Stop label ('slupek')
3. Longitude ('dlug_geo')
4. Latitude ('szer_geo')

In [2]:
params = {
        "id": "ab75c33d-3a26-4342-b36a-6e5fef0a3ac3",
        "apikey": "0cecd684-f566-4629-9dc2-c4dd5f49748f"
    }
import requests
response = requests.get("https://api.um.warszawa.pl/api/action/dbstore_get/", params=params)
data = response.json()
df_loc = []
for x in range(len(data['result'])):
    df_append = pd.DataFrame(data['result'][x]['values'])
    df_append = df_append.set_index('key')['value'].rename_axis(None)
    df_append = pd.DataFrame(df_append).T
    if x == 0:
        df_loc = df_append
    else:
        df_loc = pd.concat([df_loc, df_append])
df_loc = df_loc.reset_index()

At this stage, we collect accurate arrival time data. We send a request to the API for each stop number, stop label, and line number. Using this information, we generate timetable DataFrame, which includes:
1. Fleet number ('brygada')
2. Time ('czas')
3. Stop name ('stop_name')
4. Stop number ('stop_id')
5. Stop label ('bus_stop_nr')
6. Line number ('line')

Finally, we save this DataFrame as a 'timetable.csv' file, which is available in the repository. This file will be required for the next phase of the project.

In [16]:
timetable = []
stops = list(df_loc['zespol'].unique())
for x in stops:
    bus_stop_nrs = list(df_loc[df_loc['zespol'] == str(x)]['slupek'].unique())
    for y in bus_stop_nrs:
        with urllib.request.urlopen("https://api.um.warszawa.pl/api/action/dbtimetable_get/?id=88cd555f-6f31-43ca-9de4-66c479ad5942&busstopId={0}&busstopNr={1}&apikey=0cecd684-f566-4629-9dc2-c4dd5f49748f".format(str(x), str(y))) as url:
            data = json.loads(url.read().decode())['result']
        lines = []
        for i in data:
            lines.append(i['values'][0]['value'])
        for z in lines:
            params = {"id": "e923fa0e-d96c-43f9-ae6e-60518c9f3238",
                      "busstopId": str(x),
                      "busstopNr": str(y),
                      "line": str(z),
                      "apikey": "0cecd684-f566-4629-9dc2-c4dd5f49748f"}
            with requests.get("https://api.um.warszawa.pl/api/action/dbtimetable_get/", params=params) as response:
                res = response.json()
            df = []
            for j in list(res['result']):
                df_append = pd.DataFrame(j['values'])
                df_append = df_append.set_index('key')['value'].rename_axis(None)
                df_append = pd.DataFrame(df_append).T
                if j == list(res['result'])[0]:
                    df = df_append
                else:
                    df = pd.concat([df, df_append])
            if type(df) == list:
                break
            df = df.reset_index()[['brygada', 'czas']]
            df['stop_name'] = df_loc[(df_loc['slupek'] == str(y)) & (df_loc['zespol'] == str(x))].iloc[0]['nazwa_zespolu']
            df['stop_id'] = x
            df['bus_stop_nr'] = y
            df['line'] = z
            df['Lat'] = df_loc[(df_loc['slupek'] == str(y)) & (df_loc['zespol'] == str(x))].iloc[0]['szer_geo']
            df['Lon'] = df_loc[(df_loc['slupek'] == str(y)) & (df_loc['zespol'] == str(x))].iloc[0]['dlug_geo']
            if type(timetable) == list:
                timetable = df
            else:
                timetable = pd.concat([timetable, df])

timetable.to_csv('timetable.csv')
    

    brygada      czas     stop_name stop_id bus_stop_nr line        Lat  \
0         1  04:48:00  Płochocińska    1017          01  186  52.307248   
1         2  04:59:00  Płochocińska    1017          01  186  52.307248   
2         3  05:09:00  Płochocińska    1017          01  186  52.307248   
3         4  05:19:00  Płochocińska    1017          01  186  52.307248   
4         5  05:29:00  Płochocińska    1017          01  186  52.307248   
..      ...       ...           ...     ...         ...  ...        ...   
98       12  21:49:00  Płochocińska    1017          01  186  52.307248   
99        2  22:09:00  Płochocińska    1017          01  186  52.307248   
100      67  22:29:00  Płochocińska    1017          01  186  52.307248   
101     018  22:49:00  Płochocińska    1017          01  186  52.307248   
102      10  23:18:00  Płochocińska    1017          01  186  52.307248   

           Lon  
0    20.983925  
1    20.983925  
2    20.983925  
3    20.983925  
4    20.983925

   brygada      czas     stop_name stop_id bus_stop_nr line        Lat  \
0        1  04:48:00  Płochocińska    1017          01  186  52.307248   
1        2  04:59:00  Płochocińska    1017          01  186  52.307248   
2        3  05:09:00  Płochocińska    1017          01  186  52.307248   
3        4  05:19:00  Płochocińska    1017          01  186  52.307248   
4        5  05:29:00  Płochocińska    1017          01  186  52.307248   
..     ...       ...           ...     ...         ...  ...        ...   
12      62  18:21:00  Płochocińska    1017          01  735  52.307248   
13      64  19:17:00  Płochocińska    1017          01  735  52.307248   
14      63  20:17:00  Płochocińska    1017          01  735  52.307248   
15      62  21:16:00  Płochocińska    1017          01  735  52.307248   
16      64  22:16:00  Płochocińska    1017          01  735  52.307248   

          Lon  
0   20.983925  
1   20.983925  
2   20.983925  
3   20.983925  
4   20.983925  
..        ...  

    brygada      czas     stop_name stop_id bus_stop_nr line        Lat  \
0         1  04:48:00  Płochocińska    1017          01  186  52.307248   
1         2  04:59:00  Płochocińska    1017          01  186  52.307248   
2         3  05:09:00  Płochocińska    1017          01  186  52.307248   
3         4  05:19:00  Płochocińska    1017          01  186  52.307248   
4         5  05:29:00  Płochocińska    1017          01  186  52.307248   
..      ...       ...           ...     ...         ...  ...        ...   
121      10  22:28:00  Płochocińska    1017          02  509  52.308804   
122      12  22:43:00  Płochocińska    1017          02  509  52.308804   
123      13  22:58:00  Płochocińska    1017          02  509  52.308804   
124     247  23:13:00  Płochocińska    1017          02  509  52.308804   
125       2  23:28:00  Płochocińska    1017          02  509  52.308804   

           Lon  
0    20.983925  
1    20.983925  
2    20.983925  
3    20.983925  
4    20.983925

   brygada      czas     stop_name stop_id bus_stop_nr line        Lat  \
0        1  04:48:00  Płochocińska    1017          01  186  52.307248   
1        2  04:59:00  Płochocińska    1017          01  186  52.307248   
2        3  05:09:00  Płochocińska    1017          01  186  52.307248   
3        4  05:19:00  Płochocińska    1017          01  186  52.307248   
4        5  05:29:00  Płochocińska    1017          01  186  52.307248   
..     ...       ...           ...     ...         ...  ...        ...   
30       3  20:16:00  Płochocińska    1017          04  126  52.308290   
31       4  20:46:00  Płochocińska    1017          04  126  52.308290   
32       1  21:16:00  Płochocińska    1017          04  126  52.308290   
33       3  21:51:00  Płochocińska    1017          04  126  52.308290   
34       4  22:16:00  Płochocińska    1017          04  126  52.308290   

          Lon  
0   20.983925  
1   20.983925  
2   20.983925  
3   20.983925  
4   20.983925  
..        ...  

   brygada      czas     stop_name stop_id bus_stop_nr line        Lat  \
0        1  04:48:00  Płochocińska    1017          01  186  52.307248   
1        2  04:59:00  Płochocińska    1017          01  186  52.307248   
2        3  05:09:00  Płochocińska    1017          01  186  52.307248   
3        4  05:19:00  Płochocińska    1017          01  186  52.307248   
4        5  05:29:00  Płochocińska    1017          01  186  52.307248   
..     ...       ...           ...     ...         ...  ...        ...   
0        1  24:04:00       Sagalli    2345          02  N21  52.221737   
1        2  25:03:00       Sagalli    2345          02  N21  52.221737   
2        1  26:02:00       Sagalli    2345          02  N21  52.221737   
3        2  27:02:00       Sagalli    2345          02  N21  52.221737   
4        1  28:02:00       Sagalli    2345          02  N21  52.221737   

          Lon  
0   20.983925  
1   20.983925  
2   20.983925  
3   20.983925  
4   20.983925  
..        ...  

...

   brygada      czas     stop_name stop_id bus_stop_nr line        Lat  \
0        1  04:48:00  Płochocińska    1017          01  186  52.307248   
1        2  04:59:00  Płochocińska    1017          01  186  52.307248   
2        3  05:09:00  Płochocińska    1017          01  186  52.307248   
3        4  05:19:00  Płochocińska    1017          01  186  52.307248   
4        5  05:29:00  Płochocińska    1017          01  186  52.307248   
..     ...       ...           ...     ...         ...  ...        ...   
20       1  18:30:00      Sportowa    2367          01  L22  52.167309   
21       2  19:29:00      Sportowa    2367          01  L22  52.167309   
22       2  20:44:00      Sportowa    2367          01  L22  52.167309   
23       2  22:04:00      Sportowa    2367          01  L22  52.167309   
24       2  23:49:00      Sportowa    2367          01  L22  52.167309   

          Lon  
0   20.983925  
1   20.983925  
2   20.983925  
3   20.983925  
4   20.983925  
..        ...  

   brygada      czas     stop_name stop_id bus_stop_nr line        Lat  \
0        1  04:48:00  Płochocińska    1017          01  186  52.307248   
1        2  04:59:00  Płochocińska    1017          01  186  52.307248   
2        3  05:09:00  Płochocińska    1017          01  186  52.307248   
3        4  05:19:00  Płochocińska    1017          01  186  52.307248   
4        5  05:29:00  Płochocińska    1017          01  186  52.307248   
..     ...       ...           ...     ...         ...  ...        ...   
19       1  17:25:00        Gródek    2368          02  L22  52.160216   
20       2  18:29:00        Gródek    2368          02  L22  52.160216   
21       2  19:49:00        Gródek    2368          02  L22  52.160216   
22       2  21:14:00        Gródek    2368          02  L22  52.160216   
23       2  22:54:00        Gródek    2368          02  L22  52.160216   

          Lon  
0   20.983925  
1   20.983925  
2   20.983925  
3   20.983925  
4   20.983925  
..        ...  

   brygada      czas     stop_name stop_id bus_stop_nr line        Lat  \
0        1  04:48:00  Płochocińska    1017          01  186  52.307248   
1        2  04:59:00  Płochocińska    1017          01  186  52.307248   
2        3  05:09:00  Płochocińska    1017          01  186  52.307248   
3        4  05:19:00  Płochocińska    1017          01  186  52.307248   
4        5  05:29:00  Płochocińska    1017          01  186  52.307248   
..     ...       ...           ...     ...         ...  ...        ...   
32       4  20:49:00       Juhasów    2372          01  Z42  52.194909   
33       1  21:19:00       Juhasów    2372          01  Z42  52.194909   
34       2  21:49:00       Juhasów    2372          01  Z42  52.194909   
35       3  22:19:00       Juhasów    2372          01  Z42  52.194909   
36       1  22:49:00       Juhasów    2372          01  Z42  52.194909   

          Lon  
0   20.983925  
1   20.983925  
2   20.983925  
3   20.983925  
4   20.983925  
..        ...  

   brygada      czas     stop_name stop_id bus_stop_nr line        Lat  \
0        1  04:48:00  Płochocińska    1017          01  186  52.307248   
1        2  04:59:00  Płochocińska    1017          01  186  52.307248   
2        3  05:09:00  Płochocińska    1017          01  186  52.307248   
3        4  05:19:00  Płochocińska    1017          01  186  52.307248   
4        5  05:29:00  Płochocińska    1017          01  186  52.307248   
..     ...       ...           ...     ...         ...  ...        ...   
38       2  21:24:00    Drohobycka    2374          02  213  52.180436   
39       3  21:53:00    Drohobycka    2374          02  213  52.180436   
40       4  22:22:00    Drohobycka    2374          02  213  52.180436   
41       5  22:52:00    Drohobycka    2374          02  213  52.180436   
42       2  23:22:00    Drohobycka    2374          02  213  52.180436   

          Lon  
0   20.983925  
1   20.983925  
2   20.983925  
3   20.983925  
4   20.983925  
..        ...  

   brygada      czas     stop_name stop_id bus_stop_nr line        Lat  \
0        1  04:48:00  Płochocińska    1017          01  186  52.307248   
1        2  04:59:00  Płochocińska    1017          01  186  52.307248   
2        3  05:09:00  Płochocińska    1017          01  186  52.307248   
3        4  05:19:00  Płochocińska    1017          01  186  52.307248   
4        5  05:29:00  Płochocińska    1017          01  186  52.307248   
..     ...       ...           ...     ...         ...  ...        ...   
79       2  23:04:00        Urocza    2378          02  183  52.242279   
80       4  23:19:00        Urocza    2378          02  183  52.242279   
81      07  23:34:00        Urocza    2378          02  183  52.242279   
82       5  23:49:00        Urocza    2378          02  183  52.242279   
83       2  24:09:00        Urocza    2378          02  183  52.242279   

          Lon  
0   20.983925  
1   20.983925  
2   20.983925  
3   20.983925  
4   20.983925  
..        ...  